# Single image analysis

In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [39]:
def resize_to_window(frame):
    frame = cv.resize(frame,(frame.shape[0]//3,frame.shape[1]//3))
    return frame

def extract_pit_contours(frame):
    """ Detect and calculate area for each pit. Thresh old of minimum detection is set to 100 pixel.
    return, the image with each pit highlighted in green with its bonding box. Also returns, it coordinates and the
    area calculated at each frame.
        => Allows the seperation between pits having a growing area (stable) and pits that have a stable area (unstable)
    """
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(3,3),0)
    _, thresh = cv.threshold(blur, 80, 255, cv.THRESH_BINARY)
    kernel = np.ones((21,21),np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)
    equ = cv.equalizeHist(blur)


    contours,_ = cv.findContours(opening, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    pit_roi = []
    areas= []
    for cnt in contours:
        area = cv.contourArea(cnt)
        if 1000000 > area >= 100: 
            cv.drawContours(frame,[cnt],-1,(0,255,0),2)
            x,y,w,h = cv.boundingRect(cnt)
            pit_roi.append((x,y,w,h))
            areas.append(area)
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv.putText(frame,f"{area:.0f}",(x,y-30),
                       cv.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)


    cv.imshow("image", resize_to_window(frame))
    return frame, pit_roi, areas

def reference_frame(pits_dict):
    """
    found the reference frame at which all the pits detected during all the video are visible
    """
    ref_frame = None
    last_areas = pits_dict["area"][len(pits_dict["frame"])-10]
    average_nbr_of_objects = round(np.mean([len(objct) for objct in pits_dict["area"]]))+1
    for frame,objct in enumerate(pits_dict["area"]):
        if len(objct) == average_nbr_of_objects:
            ref_frame = frame
            break
    return ref_frame

def idx_corresponding_pit(ref_rois,end_rois):
    """
    Variables :
        -  ref_rois = array corresponding to the rois of the first frame that has a len corresponding to the mean len observed for all frames
        -  end_rois = array corresponding to the roi at the end of the 
    Out put :
    Gives an array of the rois and their index on the end_rois corresponding to the pit that where detected at the ref rois

    Purpose : 
    This function is mandatory in order to recover pits from unwanted objects that where detected by the algorythme.
    """
        
    idx = []
    
    for roi_ref in ref_rois:
        x_ref = roi_ref[0]
        y_ref = roi_ref[1]

        delta_x = [np.abs(x_ref-x_end[0]) for x_end in end_rois]
        delta_y = [np.abs(x_ref-x_end[1]) for x_end in end_rois]

        min_x_idx = np.where(delta_x==np.min(delta_x))
        min_y_idx = np.where(delta_y==np.min(delta_y))

        if min_y_idx == min_x_idx :
            idx.append(min_y_idx)
        else :
            print(min_x_idx,min_y_idx)

    return idx
            

        

def pit_recognition(video_paths="/home/ters-user/Documents/videos/30mM.mp4"):
    """
    Labelized each pit and associate to it a value between 1 (stable pit) and 2 (unstable pit).
    returns a dictionnary storing the each frame with the coordinated of each pit and the size of each bonding box.
    Also return the label associated with each detected objects.

    INPUT : video_paths of the corrosion video
    OUTPUT : frame, (x,y,w,h),area,label with,
    
         - frame : numpy array corresponding to the image at a particular frame
         - (x,y,w,h) : numpy array giving coordinates for each objetcs. (x,y) horizontal and vertical coordinates , (w,h) width and height of the bonding box
         - area : numpy array of the same size as (x,y,w,h) with the area of each object
         - label : numpy array of the same size as (x,y,w,h) with the label (1: stable pit or  2: unstable pit) for each object
    """
    cap = cv.VideoCapture(video_paths)
    
    pit_coordinates = {"frame":[],"(x,y,w,h)":[],"area":[],"label":[]}
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_number += 1
    
        displayed_img = frame.copy()
        img,roi,area = extract_pit_contours(displayed_img)
        pit_coordinates["frame"].append(frame)
        pit_coordinates["(x,y,w,h)"].append(roi[-1:]+roi[:-1])
        pit_coordinates["area"].append(area)
    
        key = cv.waitKey(25) & 0xFF
        if key == ord('q'):
            break
    
    
    cap.release()
    cv.destroyAllWindows()
    
    average_nbr_of_objects = round(np.mean([len(objct) for objct in pit_coordinates["area"]]))+1    
    ref_array = pit_coordinates["area"][reference_frame(pit_coordinates)]
    last_areas = None
    last_areas_frame = None
    for frame, areas in enumerate(pit_coordinates["area"][::-1]):
        if len(areas) == len(ref_array):
            last_areas = areas
            last_areas_frame = len(pit_coordinates["area"])- frame
            break
    for frame,objct in enumerate(pit_coordinates["area"]):
        labels = np.zeros_like(ref_array)
        # objct = [objct[idx] for idx in idx_corresponding_pit(pit_coordinates["(x,y,w,h)"][reference_frame(pit_coordinates)]
        #                                                      ,pit_coordinates["(x,y,w,h)"][len(pit_coordinates["frame"])-10])]

        if len(objct)==len(ref_array):
            for i,area in enumerate(objct):
                if np.abs(pit_coordinates["area"][reference_frame(pit_coordinates)][i] - last_areas[i]) > 3000:
                    labels[i] = 1.0
                else :
                    labels[i] = 2.0

        pit_coordinates["label"].append(labels)

    labeled_img = pit_coordinates['frame'][len(pit_coordinates["frame"])-10]
    
    print(f"{pit_coordinates['(x,y,w,h)'][reference_frame(pit_coordinates)]} \n {pit_coordinates["label"][last_areas_frame-1]}\n {pit_coordinates["area"][last_areas_frame-1]}\n\n")
    
    for roi,label in zip(pit_coordinates['(x,y,w,h)'][reference_frame(pit_coordinates)],pit_coordinates["label"][last_areas_frame-1]):
        x,y = roi[0],roi[1]
        if label == 1:
            cv.putText(labeled_img,"STABLE",(x,y),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        elif label ==2:
            cv.putText(labeled_img,"UNSTABLE",(x,y),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    
    cv.imshow("labeled_img",resize_to_window(labeled_img))
    cv.waitKey(0)
    cv.destroyAllWindows()

    return

# Video analysis

In [ ]:
pit_recognition("/home/ters-user/Documents/videos/30mM.mp4")

[(1995, 423, 13, 13), (1442, 1957, 18, 18), (1044, 1894, 20, 24), (904, 1594, 40, 36), (1873, 1173, 14, 16), (115, 1105, 90, 90), (95, 952, 14, 16), (1823, 754, 14, 13), (1938, 705, 62, 68), (2143, 521, 13, 13), (1927, 450, 24, 23)] 
 [2. 2. 2. 2. 1. 2. 2. 1. 2. 2. 2.]
 [238.5, 315.5, 1034.0, 150.5, 9749.5, 151.0, 119.5, 6579.5, 114.0, 355.0, 114.0]




## Labelling

In [48]:
liste = [2,3,4,5]
liste[::-1]

[5, 4, 3, 2]